In [1]:
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt

In [2]:
def GetRelevantTrackingData(fname):
    DLC_track = glob.glob(path_for_DLCAnnotation + '\\'+ fname + 'DeepCut_' +  '*.h5')[0]
    temp = pd.read_hdf(DLC_track)
    DLC = temp.loc[slice(None),(slice(None),'proboscisTip')]
    DLC = DLC.droplevel(level = [0,1], axis = 1)
    return(DLC)  

In [3]:
# use step 5 in moth learning as starting point to read out all the data that is included

direc = r"../../MothLearning/dataFolders/Output/Step5_FilesWith_TrueTrialAnd_ProboscisDetect_v2/"
mothlist = glob.glob(direc + '*.csv')

In [4]:
path_for_DLCAnnotation = r"G:\My Drive\Tom-Tanvi\Shared With Bing,Tom and Tanvi\Video Analysis\DeepLabCut-ImageAnalysis\take5\outputFromDLC\VideoResults\EntireDataSet"
path_for_visit_frames = r"../../MothLearning/dataFolders/Output/Step5_FilesWith_TrueTrialAnd_ProboscisDetect_v2/"

In [5]:
# parameters used
visits = ['FirstVisit/', 'LastVisit/']
visit_num = [0, -1]

In [6]:
for i in zip(visit_num, visits):
    print(i)

(0, 'FirstVisit/')
(-1, 'LastVisit/')


In [8]:
tracks = pd.DataFrame()
for file in mothlist:
    mothID = os.path.basename(file)[:-30]
    dlc = GetRelevantTrackingData(mothID)

    path_frame_Reference = glob.glob(path_for_visit_frames + mothID + '*RawDataForExplorationTime.csv')
    Visit_info = pd.read_csv(path_frame_Reference[0])
#     Visit_info = Visit_info[['MothIN', 'MothOut']]
    Visit_info = Visit_info[['MothIN', 'MothOut','ProboscisDetect']]
    

    for visitnum, direc in zip(visit_num, visits):
        outpath = os.path.join(r"../dataFolders/PaperPipelineOutput/RawTracks/" + direc)
        
        fin = Visit_info.iloc[visitnum, 0]
        fout = Visit_info.iloc[visitnum, 2]
        if np.isnan(fout):
            fout = Visit_info.iloc[visitnum, 1]
        tracks = dlc[int(fin):int(fout)].copy()
        tracks.to_csv(outpath + mothID + '_visit_' + str(visitnum) + '.csv')
        
    del dlc